In [1]:
import pandas as pd
import numpy as np
import awkward as ak

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.patches import RegularPolygon
from matplotlib.colors import to_rgb, to_rgba

import plotly as ply
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def set_custom_alpha(col_, alpha_):
    rgb_ = to_rgba(col_)
    return (col_[0], col_[1], col_[2], alpha_)

def rgb2rgba(col_):
    _ = []
    for c in col_:
        _.append(float(c)/255.0)
    _.append(1.0)
    return tuple(_)

def getNcols(N=3, cmap_='plasma'):
    cmap = plt.get_cmap(cmap_)
    cols = cmap.colors
    arr = []
    for i in range(N):
        arr.append(cols[int(256*float(i)/float(N))])
    return arr

In [3]:
def rotate_vector_2d(vec_, angle_, direction_):
    '''
    direction: +1 for counter-clockwise
               -1 for clockwise
    '''
    rmatrix = np.array([
        [np.cos(angle_), direction_*np.sin(angle_)],
        [direction_*np.sin(angle_), np.cos(angle_)]
    ])
    
    return np.dot(rmatrix, vec_)

def get_hexagon_vertices(center_, radius_, orientation_):
    '''
    orientation takes a set of 2 angle (theta, phi)
    theta, phi: angles specifying the orientation of the 
    vector normal to the surface in spherical coordinates;
    theta corrsponding to the roation about z-axis and
    phi corresponding to the rotation x-y plane
    default orientation: theta=0, phi=0
    '''
    vertices = []
    vec = np.array([0, 1])
    
    theta_ = orientation_[0]
    phi_ = orientation_[1]
    
    '''
    Add spherical roations and get the 3d vertices
    '''
    vertices_3d = []
    for ivert in range(6):
        vertex_phi_angle = ivert*np.pi/3
        vertices_3d.append(np.array([radius_*np.cos(theta_)*np.cos(phi_+vertex_phi_angle),
                                     radius_*np.cos(theta_)*np.sin(phi_+vertex_phi_angle),
                                     radius_*np.sin(theta_)]))
    
    return [ translate_vector(v, center_) for v in vertices_3d ]

def get_hexagonal_grid(center_, radius_, N):
    
    zarray = []
    
    xmin = -np.sqrt(3)*radius_
    xmax = +np.sqrt(3)*radius_
    ymin = -np.sqrt(3)*radius_
    ymax = +np.sqrt(3)*radius_
    
    X = np.linspace(xmin, xmax, N)
    Y = np.linspace(ymin, ymax, N)
    
    for x_ in X:
        for y_ in Y:
            if (np.abs(y_)>(radius_*np.sqrt(3)*0.5)) \
            or (np.abs(y_)>np.sqrt(3)*(radius_-np.abs(x_))):
                zarray.append(np.nan)
            else: zarray.append(center_[2])
    
    return X+center_[0], Y+center_[1], zarray

In [8]:
%matplotlib inline
pio.renderers.default='notebook'

In [4]:
radius = 0.5626

In [6]:
mc_hit_map = pd.read_csv('/home/rusack/shared/hgcal_electron_files/slimmed_files/sim_config22_pdgID11_beamMomentum100_slimmed_hit_map.csv',
                         nrows=1e5, usecols=['event', 'rechit_x', 'rechit_y', 'rechit_z', 'rechit_energy'])
df = mc_hit_map[mc_hit_map['event']==10]

In [9]:
df = df[df['rechit_energy']>0.5]
hit_x = df['rechit_x'].values
hit_y = df['rechit_y'].values
hit_z = df['rechit_z'].values
layer_pos = df['rechit_z'].unique()

In [10]:
N=20
X, Y, Z = get_hexagonal_grid([hit_x[4], hit_y[4], hit_z[4]], radius, N)

max_e = df['rechit_energy'].max()
e_array = [ int(e*99./max_e) for e in df['rechit_energy'].to_numpy() if e>0.5 ]

palette = getNcols(100)
c_array = [ 'rgb({}, {}, {})'.format(palette[e][0], palette[e][1], palette[e][2]) for e in e_array]

In [36]:
layout = go.Layout(
    width=1024,
    height=1024,
    scene=dict(camera=dict(eye=dict(x=1.15, y=1.15, z=0.8)), #the default values are 1.25, 1.25, 1.25
           xaxis=dict(range=(-10, 10),
                     showgrid=False,
                     zeroline=False,
                     showline=False),
           yaxis=dict(range=(-10, 10)),
           zaxis=dict(range=(0, 100)),
           aspectmode='data', #this string can be 'data', 'cube', 'auto', 'manual'
           #a custom aspectratio is defined as follows:
#           aspectratio=dict(x=1, y=1, z=5.0)
           )
)

fig = go.Figure(layout=layout)
df = df[df['rechit_energy']>0.5]
hit_x = df['rechit_x'].values
hit_y = df['rechit_y'].values
hit_z = df['rechit_z'].values
N=20
for ihit in range(len(hit_x)):
    #if ihit!=200: continue
    X, Y, Z = get_hexagonal_grid([hit_x[ihit], hit_y[ihit], hit_z[ihit]], radius, N)
    surf  = go.Surface(x=X,
                   y=Y,
                   z=np.array(Z).reshape((N, N)),
                   colorscale=[[0, c_array[ihit]], [1, c_array[ihit]]],
                   showscale=False,
                   opacity=0.5)
    #fig = go.Figure(data=[surf])
    fig.add_trace(surf)

for layer_ in layer_pos:
    surf  = go.Surface(x=np.linspace(-10, 10, N),
                   y=np.linspace(-10, 10, N),
                   z=np.ones((N, N))*layer_,
                   colorscale=['black', 'black'],
                   showscale=False,
                   opacity=0.05)
    fig.add_trace(surf)

fig.show(renderer='colab')